In [1]:
import os
import glob
import uuid
import datetime
import warnings

from itertools import product
from multiprocessing import Pool

import tqdm
import pyart
import netCDF4
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl

import grid

warnings.simplefilter('ignore')

In [2]:
def update_metadata(gnrl_meta):
    '''
    Correct general metadata dictionnary.
    
    Parameters:
    ===========
    gnrl_meta: dict
        General metadata dictionnary.
        
    Returns:
    ========
    gnrl_meta: dict
        Updated general metadata dictionnary.
    '''
    maxlon = '132.3856852067545'
    minlon = '129.70320368213441'
    maxlat = '-10.941777804922253'
    minlat = '-13.552905831511362'
    latres = '0.0225'

    origin_altitude = '50'
    origin_latitude = '-12.249'
    origin_longitude = '131.044'
    projection = 'Azimuthal equidistant projection'

    obsolete_keys = ['original_format', 'n_gates_vary', 'driver', 'start_datetime', 
                     'start_time', 'end_datetime', 'end_time', 'scan_name', 'scan_id', 
                     'ray_times_increase', 'Conventions', 'Sub_conventions']
    for key in obsolete_keys:
        try:
            gnrl_meta.pop(key)
        except KeyError:
            pass

    gnrl_meta['version'] = '2018.06_level2'
    gnrl_meta['created'] = datetime.datetime.now().isoformat()
    gnrl_meta['uuid'] = str(uuid.uuid4())
    gnrl_meta['processing_level'] = 'L2'

    gnrl_meta['geospatial_bounds'] = f"({minlon}, {maxlon}, {minlat}, {maxlat})"
    gnrl_meta['geospatial_lat_min'] = minlat
    gnrl_meta['geospatial_lat_max'] = maxlat
    gnrl_meta['geospatial_lat_units'] = "degrees_north"
    gnrl_meta['geospatial_lat_resolution'] = latres
    gnrl_meta['geospatial_lon_min'] = minlon
    gnrl_meta['geospatial_lon_max'] = maxlon
    gnrl_meta['geospatial_lon_units'] = "degrees_east"
    gnrl_meta['geospatial_lon_resolution'] = latres
    gnrl_meta['geospatial_vertical_min'] = '0'
    gnrl_meta['geospatial_vertical_max'] = '20000'
    gnrl_meta['geospatial_vertical_resolution'] = '500'
    gnrl_meta['geospatial_vertical_units'] = "meters"
    gnrl_meta['origin_latitude'] = origin_latitude
    gnrl_meta['origin_longitude'] = origin_longitude
    gnrl_meta['origin_altitude'] = origin_altitude
    gnrl_meta['geospatial_projection'] = projection
    
    return gnrl_meta

In [3]:
def read_metadata(infile):
    with netCDF4.Dataset(infile, 'r') as ncid:
        metadata = dict()
        for k in ncid.ncattrs():
            metadata[k] = getattr(ncid, k)
    return metadata

In [4]:
def compute_eth(x, y, infile, ETH_TH=0):
    """
    Compute the Echo Top Height on a grid
    
    Parameters
    ==========
    x: ndarray <x>
        x-axis coordinates of the output grid
    y: ndarray <y>
        y-axis coordinates of the output grid
    radar: Py-ART radar structure
        Input radar data 
    ETH_TH: float
        Reflectivity threshold for which we want to compute the echo top height.
        
    Returns:
    ========
    eth: ndarray <x, y>
        Echo top height on a grid of dimension (x, y).        
    """
    # Reading data.
#     r = radar.range['data']
#     refl = radar.fields['reflectivity']['data'].copy()
    
#     sw = radar.get_slice(0)    
#     refl_ref = refl[sw]
#     azi_ref = radar.azimuth['data'][sw]
#     azi_level_0 = radar.azimuth['data'][sw]    
#     elev_ref = radar.elevation['data'][sw].mean()
    
#     x_radar, y_radar, _ = radar.get_gate_x_y_z(0)
    
#     R, A = np.meshgrid(r, azi_level_0)    
#     try:
#         zhrad = radar.altitude['data'].mean()
#     except Exception:
#         zhrad = 50 

    with netCDF4.Dataset(infile, 'r') as ncid:
        r = ncid['range'][:].filled(np.NaN)
        azimuth = ncid['azimuth'][:].filled(np.NaN)
        elevation = ncid['elevation'][:].filled(np.NaN)
        refl = ncid['reflectivity'][:].filled(np.NaN)
        st_sweep = ncid['sweep_start_ray_index'][:]
        ed_sweep = ncid['sweep_end_ray_index'][:]

    sw = slice(st_sweep[0], ed_sweep[0])
    refl_ref = refl[sw]
    azi_ref = azimuth[sw]
    azi_level_0 = azimuth[sw]
    elev_ref = elevation[sw]
    x_radar, y_radar, _ = pyart.core.transforms.antenna_vectors_to_cartesian(r, azimuth[sw], elevation[sw])
    
    # CPOL altitude in m
    zhrad = 50  
    
    R, A = np.meshgrid(r, azi_level_0) 

    # Theta 3dB angle.
    da = azi_level_0[1] - azi_level_0[0]    
    if np.abs(1 - da) < np.abs(1.5 - da):
        th3 = 1
    else:
        th3 = 1.5
    
    # Initializing array.
    eth = np.zeros(refl_ref.shape) + np.NaN    
    
    # Looping over slices
    for sts, eds in zip(st_sweep[1:], ed_sweep[1:]):
        sl = slice(sts, eds)
        
        refl_iter = refl[sl]
        azi_iter = azimuth[sl]
        elev_iter = elevation[sl].mean()
        if elev_iter == 90:
            continue       
            
        eth = grid.echo_top_height(r, azi_level_0, azi_ref, azi_iter, refl_ref,
                                   refl_iter, elev_ref, elev_iter, 0.5, 0, eth)

        refl_ref = refl_iter.copy()
        azi_ref = azi_iter.copy()
        elev_ref = elev_iter
        
    # Turning elevation angle into heights.
    cloudtop = R * np.sin(eth * np.pi / 180) + zhrad
    
    # Initializing return array and gridding it.
    eth = np.zeros((len(x), len(y)))
    eth = grid.grid_radar(cloudtop, x, y, x_radar, y_radar, th3)

    return eth

In [5]:
def get_file_list(input_dir):
    '''
    Generate file list.
    
    Parameter:
    ===========
    input_dir: str
        Path to input data directory
        
    Returns:
    ========
    flist: list
        Input file list.
    '''
    flist = sorted(glob.glob(os.path.join(input_dir, '*.nc')))
    if len(flist) == 144:
        return flist
    
    flist = [None] * 144
    for cnt, (h, m) in enumerate(product(range(0, 24), range(0, 6))):
            infiles = glob.glob(os.path.join(input_dir, f'*_{h:02}{m}*.nc'))
            if len(infiles) == 0:
                continue

            infile = infiles[0]
            if os.path.isfile(infile):
                flist[cnt] = infile
    
    return flist

In [6]:
def mkdir(mydir):
    if os.path.exists(mydir):
        return None
    
    try:
        os.mkdir(mydir)
    except FileExistsError:
        return None
    
    return None        

In [12]:
def make_dailys(inargs):
    
    x, y, date, INDIR, OUTDIR, MOMENT_NAME = inargs
    
    # Check if input dir exits.
    indir = os.path.join(INDIR, str(date.year), date.strftime('%Y%m%d'))
    if not os.path.exists(indir):
#         print(f'Input dir {indir} does not exist.')
        return None
    flist = get_file_list(indir)
    
    # Generate output file name
    outdir = os.path.join(OUTDIR, MOMENT_NAME.upper())
    mkdir(outdir)
    outfilename = os.path.join(outdir, 'CPOL_{}_{}.nc'.format(MOMENT_NAME.upper(), date.strftime('%Y%m%d')))
    if os.path.exists(outfilename):
        print(f'Output file {outfilename} already exists. Doing nothing.')
        return None
    
    XDIM = len(x)
    YDIM = len(y)
    
    X, Y = np.meshgrid(x, y)
    R = np.sqrt(X ** 2 + Y ** 2)
    
    # Read data
    IS_FILE = np.zeros((144), dtype=np.int32) + 1
    RAIN_TOT = np.zeros((144, XDIM, YDIM))
    for cnt, infile in enumerate(flist):
        if infile is None:
            RAIN_TOT[cnt, :, :] = np.NaN
            IS_FILE[cnt] = 0
            continue
            
        if 'gnrl_meta' not in locals():
            gnrl_meta = read_metadata(infile)
            
        eth = compute_eth(x, y, infile, 0)
                
        eth[R >= RMAX] = np.NaN
        RAIN_TOT[cnt, :, :] = eth.copy()
        
    RAIN_TOT = np.ma.masked_where(np.isnan(RAIN_TOT), RAIN_TOT)
    gnrl_meta = update_metadata(gnrl_meta)   
    rain_meta = {'standard_name': 'echo_top_height', 'long_name': 'Echo top height', 'units':'meters', 
                 'description': '0dB echo top height using Lakshman et al. 2013 algorithm.'}
    
    # Generate time dimension
    st = np.array(date, dtype=np.datetime64)
    ed = np.array(date, dtype=np.datetime64) +  np.timedelta64(1,'D')
    dtime = np.arange(st, ed, np.timedelta64(10, 'm'))  # Every 10 minutes
    time_unit = f'seconds since {str(dtime[0])}'
    time = netCDF4.date2num(dtime.tolist(), time_unit).astype(np.int32)
    
    # Write data
    with netCDF4.Dataset(outfilename, 'w') as ncid:
        ncid.createDimension('time', 144)
        ncid.createDimension("x", XDIM)
        ncid.createDimension("y", YDIM)

        mymoment = ncid.createVariable(MOMENT_NAME, RAIN_TOT.dtype, ("time", "x", "y"), zlib=True)
        ncquality = ncid.createVariable('isfile', IS_FILE.dtype, ("time",))

        nctime = ncid.createVariable('time', time.dtype, 'time')
        ncx = ncid.createVariable('x', x.dtype, 'x')
        ncy = ncid.createVariable('y', y.dtype, 'y')

        nctime[:] = time
        nctime.units = time_unit
        ncx[:] = x
        ncx.units = 'km'
        ncy[:] = y
        ncy.units = 'km'
        ncquality[:] = IS_FILE
        ncquality.units = ''
        ncquality.setncattr('description', "0: no data, 1: data available at this time step")
        
        # Write attributes
        mymoment[:] = RAIN_TOT
        for k, v in rain_meta.items():            
            mymoment.setncattr(str(k), str(v))

        for k, v  in gnrl_meta.items():
            ncid.setncattr(k, str(v))
            
    return None

In [13]:
XDIM = 117
YDIM = 117
RMAX = 140e3
OUTDIR = '/g/data2/rr5/vhl548/NEW_CPOL_level_2'
INDIR = '/g/data2/rr5/vhl548/CPOL_level_1b/PPI/'
MOMENT_NAME = 'echo_top_height'

x = np.linspace(-145e3, 145e3, 117)
y = np.linspace(-145e3, 145e3, 117)

In [14]:
date_range = pd.date_range('19981207', '20071231')
args_list = [None] * len(date_range)
for cnt, dt in enumerate(date_range):
    args_list[cnt] = (x, y, dt, INDIR, OUTDIR, MOMENT_NAME)    

In [ ]:
with Pool(16) as pool:
    rslt = list(tqdm.tqdm_notebook(pool.imap(make_dailys, args_list), total=len(args_list)))